In [5]:
# %pylab
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import euclidean_distances
from scipy.sparse import csr_matrix
import configparser

In [8]:
maxDistance = 1.2

In [10]:
def NNeighbor(mat):
    idx = list()
    distMatrix = euclidean_distances(mat)
    distMatrix = csr_matrix(distMatrix)
#     print(distMatrix)
    for i in range(distMatrix.shape[0]):
        row = distMatrix.getrow(i).toarray()[0].ravel()
        top_indices = row.argsort()[1]
        # print(top_indices)
        top_values = row[row.argsort()[1]]
#         print(row[row.argsort()])
        if top_values > maxDistance:
            idx.append(-1)
#         print top_indices
#         print(top_values)
        else:
            idx.append(top_indices)
    return idx

def trace( *args ):
    #pass 
    print( "".join(map(str,args)) )

In [14]:
# print(NNeighbor(np.array(unlabeled_markers)[:,0:2]))

In [13]:
unlabeled_markers = ([[1, 1, 0.1], [-2, -2, 0.2], [7, 0., 0.1], 
                     [1, 2, 0.3],[-3, -2, 0.4],[8, 1, 0.2]])

# from opti-track to map coordinate
# opti-track x z
#        map y x
n_unlabled_markers = np.array(unlabeled_markers)
# n_markers = np.vstack((n_unlabled_markers[:,2], n_unlabled_markers[:,0]))
# n_markers = n_markers.T

trace("raw markers:")
trace(n_unlabled_markers[:,0:3])

markers_index = NNeighbor(n_unlabled_markers[:,0:2])
trace("neighbor markers index:")
trace(markers_index)

markers_set = []

for i in range(len(markers_index)):
    if(markers_index[i] > i):
        markers_set.append([i, markers_index[i]])


n_markers_set = np.array(markers_set)
trace("markers set:")
trace(markers_set)

front_markers_set = list()
back_markers_set = list()
for i, j in markers_set:
    if n_unlabled_markers[i][2] > n_unlabled_markers[j][2]:
        front_markers_set.append(i)
        back_markers_set.append(j)
    else:
        front_markers_set.append(j)
        back_markers_set.append(i)
# print(front_markers)

back_markers = (n_unlabled_markers[back_markers_set])[:,0:2]
front_markers = (n_unlabled_markers[front_markers_set])[:,0:2]
trace("back markers:")
trace(back_markers)
trace("font markers:")
trace(front_markers)

robot_location = (back_markers+front_markers)/2
trace("robot location:")
trace(robot_location)

delta_points = front_markers - back_markers
robot_theta = (np.arctan2(delta_points[:,1], delta_points[:,0])+np.pi*2) % (np.pi*2)
trace("robot theta:")
trace(robot_theta)

print("robot_data:")
print(np.hstack((robot_location, robot_theta.reshape(robot_theta.shape[0],1))))

raw markers:
[[ 1.   1.   0.1]
 [-2.  -2.   0.2]
 [ 7.   0.   0.1]
 [ 1.   2.   0.3]
 [-3.  -2.   0.4]
 [ 8.   1.   0.2]]
neighbor markers index:
[3, 4, -1, 0, 1, -1]
markers set:
[[0, 3], [1, 4]]
back markers:
[[ 1.  1.]
 [-2. -2.]]
font markers:
[[ 1.  2.]
 [-3. -2.]]
robot location:
[[ 1.   1.5]
 [-2.5 -2. ]]
robot theta:
[ 1.57079633  3.14159265]
robot_data:
[[ 1.          1.5         1.57079633]
 [-2.5        -2.          3.14159265]]
